In [1]:
from datasets import load_dataset

ds = load_dataset("jonathanli/law-stack-exchange")

/Users/adrienloizeau/opt/anaconda3/envs/lerobot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
split = ["train", "test", "validation"]

full_dataset = ""
for s in split:  
    for i in range(len(ds[s]["title"])):  
        title = ds[s]["title"][i]
        body = ds[s]["body"][i]
        full_dataset += title + "\n" + body + "\n\n" 

full_dataset[:100]

'Why TVs / laptops etc. do not warn about too brirght screen whereas many smartphones warn about too '

In [3]:
chars = sorted(list(set(full_dataset)))
vocab_size = len(chars)

print("".join(chars))
print(vocab_size)

	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ £§©®´·¸ßàäçèéêöüıł  ​–—‘’“”•… €₹™⇒≠ﬀ
134


In [4]:
unecessary_chars  = ["©","®","₹","™","–—","⇒", "…"]
filtered_dataset  = full_dataset
for u in unecessary_chars:
    filtered_dataset = filtered_dataset.replace(u,"")

chars = sorted(list(set(filtered_dataset)))
vocab_size = len(chars)

print("".join(chars))
print(vocab_size)

dataset = filtered_dataset

	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ £§´·¸ßàäçèéêöüıł  ​–—‘’“”• €≠ﬀ
128


Tokenise my data

In [ ]:
encode_dict = { c:i for i,c in enumerate(chars)}
decode_dict = { i:c for i,c in enumerate(chars)}


encode = lambda x: [encode_dict[c] for c in x]
decode = lambda y: ''.join([decode_dict[c] for c in y])

encoded_data = encode(dataset[:10])

print(dataset[:10])
print(encoded_data)
decode(encoded_data)

Why TVs / 
[57, 74, 91, 2, 54, 56, 85, 2, 17, 2]


'Why TVs / '

In [7]:
# from tokenizers import Tokenizer, models, trainers
# from tokenizers.pre_tokenizers import Whitespace
# tokenizer.pre_tokenizer = Whitespace()
# tokenizer = Tokenizer(models.BPE())

# trainer = trainers.BpeTrainer(vocab_size=256)

# tokenizer.train_from_iterator([dataset], trainer=trainer)
# tokenizer.save("custom_tokenizer.json")  # Save
# tokenizer = Tokenizer.from_file("custom_tokenizer.json")  # Load

Dataloader

In [18]:
import time 

# basic structure
for i in range(len(dataset[:5])+1):
    time.sleep(1)
    print("context - ", dataset[:i])
    print("to predict - ", dataset[i+1])

context -  
to predict -  h
context -  W
to predict -  y
context -  Wh
to predict -   
context -  Why
to predict -  T
context -  Why 
to predict -  V
context -  Why T
to predict -  s


In [50]:
# block size introduction
block_size = 8 

x = encode(dataset[:block_size]) # context
y =  encode(dataset[1:block_size +1]) # to predict
for t in range(block_size):
    time.sleep(0.5)
    print("context - ", x[:t+1])
    print("to predict - ", y[t])

context -  [57]
to predict -  74
context -  [57, 74]
to predict -  91
context -  [57, 74, 91]
to predict -  2
context -  [57, 74, 91, 2]
to predict -  54
context -  [57, 74, 91, 2, 54]
to predict -  56
context -  [57, 74, 91, 2, 54, 56]
to predict -  85
context -  [57, 74, 91, 2, 54, 56, 85]
to predict -  2
context -  [57, 74, 91, 2, 54, 56, 85, 2]
to predict -  17


In [ ]:
# adding batch dimensions

import torch 
import random

batch_size = 4 # number of data loaded in parallel
block_size = 8

def get_batch_v1():
    data = torch.tensor(encode(dataset))
    idx = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i-block_size: i] for i in idx])
    y = torch.stack([data[i] for i in idx])
    return x, y
for i in range(1):
    print(get_batch())



(tensor([[75, 80,  2, 53, 83, 87, 67, 84],
        [88, 81, 75, 70, 17, 84, 71, 85],
        [ 2,  1,  1,  1, 54, 74, 75, 85],
        [81, 73, 71, 80, 71, 84,  2, 38]]), tensor([71, 69,  2, 67]))


Data split

In [80]:
# data split 
split = int(len(dataset)*0.9)
data_train = encode(dataset[:split]) # 90% of the dataset used for training
data_val = encode(dataset[split:]) # 10% for validation

print(data_train[:20])
print(decode(data_train[:20]))

data_train = torch.tensor(data_train)
data_val = torch.tensor(data_val)

[57, 74, 91, 2, 54, 56, 85, 2, 17, 2, 78, 67, 82, 86, 81, 82, 85, 2, 71, 86]
Why TVs / laptops et


In [ ]:
# Adding split 

import torch 
import random

batch_size = 4 # number of data loaded in parallel
block_size = 8

def get_batch(split):
    data = data_train if split == "train" else data_val
    idx = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i-block_size: i] for i in idx])
    y = torch.stack([data[i] for i in idx])
    return x, y

for i in range(1):
    print(get_batch("train"))


(tensor([[71, 79, 67, 84, 77,  2, 81, 72],
        [82, 78, 81, 91, 79, 71, 80, 86],
        [79, 67, 86, 75, 81, 80,  2, 86],
        [75, 69, 71, 80, 85, 75, 80, 73]]), tensor([ 2, 33, 81, 14]))
